In [4]:
import gc
gc.collect()    # fuerza que se elimine informacion que no es relevante

86

### H2O

https://www.cienciadedatos.net/documentos/py04_machine_learning_con_h2o_y_python

In [5]:
!pip3 install h2o

^C

You should consider upgrading via the 'python -m pip install --upgrade pip' command.Requirement already satisfied: requests in c:\users\usuario\appdata\local\programs\python\python37\lib\site-packages (from h2o) (2.25.1)



In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error

import h2o
from h2o.automl import H2OAutoML

In [3]:
train = pd.read_csv('diamonds_train.csv')
test = pd.read_csv('diamonds_test.csv')

In [8]:
train

,Unnamed: 0,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.21,Ideal,H,VS2,63.0,57.0,6.73,6.70,4.23,6134
1,1,0.28,Very Good,D,VVS2,64.0,56.0,4.14,4.17,2.66,532
2,2,0.42,Premium,F,VS1,61.2,58.0,4.86,4.82,2.96,1103
3,3,0.26,Ideal,H,IF,61.1,57.0,4.16,4.12,2.53,600
4,4,1.10,Good,G,SI1,63.4,57.0,6.52,6.55,4.14,4997
...,...,...,...,...,...,...,...,...,...,...,...
40340,40340,1.55,Premium,H,VS2,61.3,61.0,7.46,7.39,4.55,11708
40341,40341,0.36,Ideal,D,SI1,60.6,56.0,4.58,4.63,2.79,619
40342,40342,0.57,Very Good,I,VS2,62.2,55.0,5.33,5.34,3.32,1267
40343,40343,1.01,Very Good,F,IF,59.6,62.0,6.47,6.56,3.88,9965


In [3]:
# hay que instalar JAVA versión < 14
# https://www.oracle.com/java/technologies/javase-jdk11-downloads.html

In [6]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.11+9-LTS-194, mixed mode)
  Starting server from C:\Users\Usuario\AppData\Local\Programs\Python\Python37\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Usuario\AppData\Local\Temp\tmpkw07p3_4
  JVM stdout: C:\Users\Usuario\AppData\Local\Temp\tmpkw07p3_4\h2o_Usuario_started_from_python.out
  JVM stderr: C:\Users\Usuario\AppData\Local\Temp\tmpkw07p3_4\h2o_Usuario_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,1 month and 6 days
H2O_cluster_name:,H2O_from_python_Usuario_21z13p
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.855 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"accepting new members, healthy"


In [9]:
test

,Unnamed: 0,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.30,Ideal,H,SI2,60.0,56.0,4.41,4.43,2.65
1,1,0.34,Ideal,D,IF,62.1,57.0,4.52,4.46,2.79
2,2,1.57,Very Good,I,VS2,60.3,58.0,7.58,7.55,4.56
3,3,0.31,Ideal,H,VS2,61.8,57.0,4.32,4.36,2.68
4,4,1.51,Good,I,VVS1,64.0,60.0,7.26,7.21,4.63
...,...,...,...,...,...,...,...,...,...,...
13444,13444,1.05,Ideal,F,SI2,60.3,55.0,6.55,6.68,3.99
13445,13445,0.27,Very Good,G,VS1,61.9,56.0,4.13,4.16,2.56
13446,13446,1.50,Premium,D,VS2,61.8,60.0,7.30,7.37,4.53
13447,13447,1.70,Premium,F,SI2,60.2,61.0,7.79,7.68,4.66


In [7]:
h2train = h2o.H2OFrame(train)
h2test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
columnas = [a for a in h2train.columns if a != "price"][1:]
x = columnas
y = "price"
print(columnas)

['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']


In [7]:
automl = H2OAutoML(max_models=50, seed=42, max_runtime_secs=300, sort_metric='RMSE')
automl.train(x=x, y=y, training_frame=h2train)
# en Windows no está disponible XGBoost :'(

AutoML progress: |
21:26:07.766: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████| 100%


In [8]:
print('[INFO] Models leader board:')
leader_board = automl.leaderboard
leader_board.head()

[INFO] Models leader board:


model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20210620_212607,524.572,275176,275176,265.168,0.0938705
StackedEnsemble_BestOfFamily_AutoML_20210620_212607,531.021,281984,281984,269.146,0.0967715
GBM_grid__1_AutoML_20210620_212607_model_10,532.143,283176,283176,272.761,0.099033
GBM_4_AutoML_20210620_212607,532.776,283850,283850,267.663,0.0924966
GBM_3_AutoML_20210620_212607,533.556,284682,284682,272.097,0.0980888
GBM_grid__1_AutoML_20210620_212607_model_5,534.292,285468,285468,272.373,nan
GBM_2_AutoML_20210620_212607,534.413,285598,285598,275.044,nan
GBM_grid__1_AutoML_20210620_212607_model_2,537.035,288407,288407,270.383,0.0934097
GBM_grid__1_AutoML_20210620_212607_model_9,538.6,290089,290089,273.964,nan
GBM_grid__1_AutoML_20210620_212607_model_7,542.448,294250,294250,278.287,nan


In [9]:
predictions = automl.leader.predict(h2test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [10]:
test.rename(columns={"Unnamed: 0" : "id"}, inplace = True)

In [11]:
test["price"] = predictions.as_data_frame()
columnasentrega = ["id","price"]
entrega = test["price"]
test = test[columnasentrega]

In [12]:
test.head()

,id,price
0,0,414.954507
1,1,1699.129208
2,2,9550.341099
3,3,535.114304
4,4,9832.887516


In [13]:
test['price'].astype("int")

0          414
1         1699
2         9550
3          535
4         9832
         ...  
13444     4808
13445      457
13446    14941
13447    10794
13448      739
Name: price, Length: 13449, dtype: int32

In [14]:
test.to_csv('to_submit_int.csv', index=False)

In [15]:
testing = pd.read_csv('to_submit_int.csv')
testing

,id,price
0,0,414.954507
1,1,1699.129208
2,2,9550.341099
3,3,535.114304
4,4,9832.887516
...,...,...
13444,13444,4808.840639
13445,13445,457.755320
13446,13446,14941.798953
13447,13447,10794.781777


In [16]:
h2o.cluster().shutdown()

H2O session _sid_84bf closed.


**Evaluación con los datos reales**

In [17]:
true = pd.read_csv('evaluation_price.csv')

y_pred = testing['price']
y_true = true['price']
np.sqrt(mean_squared_error(y_true, y_pred))

526.3573341009826